<table>
 <tr align=left><td><img align=left src="./images/CC-BY.png">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license. (c) Kyle T. Mandli</td>
</table>

Note:  This material largely follows the text "Numerical Linear Algebra" by Trefethen and Bau (SIAM, 1997) and is meant as a guide and supplement to the material presented there.

In [ ]:
%matplotlib inline
from __future__ import print_function
import numpy
import matplotlib.pyplot as plt

# QR Factorizations and Least Squares

## Projections

A **projector** is a square matrix $P$ that satisfies
$$
    P^2 = P.
$$

Why does this definition make sense?  Why do we require it to be square?

A projector comes from the idea that we want to project a vector $v$ onto a lower dimensional subspace.  Of course if $v$ lies completely within this subspace, i.e. $v \in \text{range}(P)$ then $P v = v$.  This motivates the definition above.

If $Pv = v'$ then $v'$ should already be in the sub-space so that subsequent applications of $P$ should result in no change, i.e.
$$
    P( Pv ) = Pv' = v'.
$$

As another example, take a vector $x \notin \text{range}(P)$ and project it onto the subspace $Px = v$.  If we apply the projection again to $v$ now we have

$$\begin{aligned}
    Px &= v \\
    P^2 x & = Pv = v \Rightarrow \\
    P^2 &= P.
\end{aligned}$$

It is also important to keep in mind the following, given again $x \notin \text{range}(P)$, if we look at the difference between the projection and the original vector $Px - x$ and apply the projection again we have
$$
    P(Px - x) = P^2 x - Px = 0
$$
which means the difference between the projected vector $Px = v$ lies in the null space of $P$, $v \in \text{null}(P)$.

### Complementary Projectors

A projector also has a complement defined as $I - P$.  

Show that this complement is also a projector.

We can show that again this a projector as

$$\begin{aligned}
    (I - P)^2 &= I - IP - IP + P^2 \\
    &= I - 2 P + P^2 \\
    &= I - 2P + P \\
    &= I - P.
\end{aligned}$$

It turns out that the complement projects exactly onto $\text{null}(P)$.  

Take 
$$
    x \in \text{null}(P),
$$ 
then 
$$
    (I - P) x = x - P x = x
$$ 
since $P x = 0$ implying that $x \in \text{range}(I - P)$.

We also know that 
$$
    (I - P) x \in \text{null}(P)
$$ 
as well.  

This shows that the 
$$
    \text{range}(I - P) \subseteq \text{null}(P)
$$ 
and 
$$
    \text{range}(I - P) \supseteq \text{null}(P)
$$ 
implying that 
$$
    \text{range}(I - P) = \text{null}(P)
$$ 
exactly.  

Reflect on these subspaces and convince yourself that this all makes sense.

This result provides an important property of a projector and its complement, namely that they divide a space into two subspaces whose intersection is 
$$
    \text{range}(I - P) \cap \text{range}(P) = \{0\}
$$
or
$$
    \text{null}(P) \cap \text{range}(P) = \{0\}
$$
These two spaces are called **complementary subspaces**.

Given this property we can take any $P \in \mathbb C^{m \times m}$ which will split $\mathbb C^{m \times m}$ into two subspaces $S$ and $V$, assume that $s \in S = \text{range}(P)$, and $v \in V = \text{null}(P)$.  If we have $x \in \mathbb C^{m \times m}$ that we can split the vector $x$ into components in $S$ and $V$ by using the projections
$$\begin{aligned}
    P x = x_S& &x_s \in S \\
    (I - P) x = x_V& &x_V \in V
\end{aligned}$$
which we can also observe adds to the original vector as
$$
    x_S + x_V = P x + (I - P) x = x.
$$

Try constructing a projection matrix so that $P \in \mathbb R^3$ that projects a vector into one of the coordinate directions ($\mathbb R$).  
 - What is the complementary projector?
 - What is the complementary subspace?

### Orthogonal Projectors

An **orthogonal projector** is one that projects onto a subspace $S$ that is orthogonal to the complementary subspace $V$ (this is also phrased that $S$ projects along a space $V$).  Note that we are only talking about the subspaces (and their basis), not the projectors!

A **hermitian** matrix is one whose complex conjugate is itself, i.e.
$$
    P = P^\ast.
$$

With this definition we can then say:  *A projector $P$ is orthogonal if and only if $P$ is hermitian.*

### Projection with an Orthonormal Basis

We can also directly construct a projector that uses an orthonormal basis on the subspace $S$.  If we define another matrix $Q \in \mathbb C^{m \times n}$ which is unitary (its columns are orthonormal) we can construct an orthogonal projector as
$$
    P = Q Q^*.
$$
Note that the resulting matrix $P$ is in $\mathbb C^{m \times m}$ as we require.  This means also that the dimension of the subspace $S$ is $n$.

**Example:  Construction of an orthonormal projector**

Take $\mathbb R^3$ and derive a projector that projects onto the x-y plane and is an orthogonal projector.

$$
    Q = \begin{bmatrix} 1 & 0 \\ 
                        0 & 1 \\ 
                        0 & 0 
        \end{bmatrix}, \quad 
    Q Q^\ast = \begin{bmatrix} 1 & 0 \\ 
                               0 & 1 \\ 
                               0 & 0 
               \end{bmatrix} 
               \begin{bmatrix} 1 & 0 & 0 \\
                               0 & 1 & 0 
               \end{bmatrix} = \begin{bmatrix} 
                               1 & 0 & 0 \\
                               0 & 1 & 0 \\ 
                               0 & 0 & 0 
               \end{bmatrix}
$$

In [ ]:
Q = numpy.array([[1, 0],[0, 1],[0, 0]])
P = numpy.dot(Q, numpy.conjugate(numpy.transpose(Q)))
I = numpy.identity(3)

x = numpy.array([3, 4, 5])
x_S = numpy.dot(P, x)
x_V = numpy.dot(I - P, x)
print(x)
print(x_S)
print(x_V)
print(x_S + x_V)

#### Example: Construction of a projector that eliminates a direction

Goal:  Eliminate the component of a vector in the direction $q$.

Form the projector $P = q q^\ast \in \mathbb C^{m \times m}$.  The complement $I - P$ will then include everything **BUT** that direction.  If $||q|| = 1$ we can then simply use $I - q q^\ast$.  If not we can write the projector in terms of the arbitrary vector $a$ as
$$
    I - \frac{a a^\ast}{||a||} = I - \frac{a a^\ast}{a^\ast a}
$$
Note that differences in the resulting dimensions between the two values in the fraction.  Also note that as we saw with the outer product, the resulting $\text{rank}(a a^\ast) = 1$.

Now again try to construct a projector in $\mathbb R^3$ that projects onto the $x$-$y$ plane.

In [ ]:
q = numpy.array([0, 0, 1])
P = numpy.outer(q, q.conjugate())
P_comp = numpy.identity(3) - P

x = numpy.array([3, 4, 5])
print(numpy.dot(P, x))
print(numpy.dot(P_comp, x))

a = numpy.array([0, 0, 3])
P = numpy.outer(a, a.conjugate()) / (numpy.dot(a, a.conjugate()))
P_comp = numpy.identity(3) - P
print(numpy.dot(P, x))
print(numpy.dot(P_comp, x))

## QR Factorization

One of the most important ideas in linear algebra is the concept of factorizing an original matrix into different constituents that may have useful properties.  These properties can help us understand the matrix better and lead to numerical methods.  In numerical linear algebra one of the most important factorizations is the **QR factorization**.  

The basic idea is that we want to break up $A$ into its successive spaces spanned by the columns of $A$.  If we have
$$
    A = \begin{bmatrix}  &  &  \\  &  &  \\ a_1 & \cdots & a_n \\  &  &  \\  &  &  \end{bmatrix}
$$
then we want to construct the sequence
$$
    \text{span}(a_1) \subseteq \text{span}(a_1, a_2) \subseteq \text{span}(a_1, a_2, a_3) \subseteq \cdots \subseteq \text{span}(a_1, a_2, \ldots , a_n)
$$
where here $\text{span}(v_i)$ indicates the subspace spanned by the vectors $v_i$.  

QR factorization attempts to construct a set of orthonormal vectors $q_i$ that span each of the subspaces, i.e. 
$$
    \text{span}(a_1, a_2, \ldots, a_j) = \text{span}(q_1, q_2, \ldots, q_j).
$$

Lets consider this for the first few vectors $q_i$.
1. For $\text{span}(a_1)$ we can directly use $a_1$ but normalize the vector such that
$$
    q_1 = \frac{a_1}{||a_1||}.
$$

2. For $\text{span}(a_1, a_2)$ we already have $q_1$ so we need to have a vector $q_2$ that is orthogonal to $q_1$, i.e.
$$
    \langle q_1, q_2 \rangle = q_1^* q_2 = 0
$$
and is again normalized.  We can accomplish this by modifying $a_2$ such that
$$
    q_2 = \frac{a_2 - \langle q_1, a_2\rangle}{||a_2 - \langle q_1, a_2\rangle||}.
$$
which we can show is orthogonal to $q_1$ as
$$\begin{aligned}
    \langle q_1, q_2 \rangle &= \left(\langle  q_1, a_2 - \langle  q_1, a_2 \rangle \rangle \right) \frac{1}{||a_2 - \langle  q_1, a_2 \rangle||} \\
    &= \left(\langle  q_1, a_2 \rangle - \langle  q_1, a_2 \rangle\right) \frac{1}{||a_2 - \langle  q_1, a_2 \rangle||} \\
    &= 0.
\end{aligned}$$

These results suggest then that we may have a matrix factorization that has the following form:
$$
    \begin{bmatrix}  &  &  \\  &  &  \\ a_1 & \cdots & a_n \\  &  &  \\  &  &  \end{bmatrix} = 
    \begin{bmatrix}  &  &  \\  &  &  \\ q_1 & \cdots & q_n \\  &  &  \\  &  &  \end{bmatrix}
    \begin{bmatrix} r_{11} & r_{12} & \cdots & r_{1n} \\  & r_{22} &  &  \\  &  & \ddots & \vdots \\  &  &  & r_{nn} \end{bmatrix}.
$$
If write out as a matrix multiplication we have
$$\begin{aligned}
    a_1 &= r_{11} q_1 \\
    a_2 &= r_{22} q_2 + r_{12} q_1 \\
    a_3 &= r_{33} q_3 + r_{23} q_2 + r_{13} q_1 \\
    &\vdots
\end{aligned}$$
we can also identify at least the first couple of value of $r$ as
$$\begin{aligned}
    r_{11} &= \left \Vert a_1 \right \Vert \\
    r_{12} &= \langle q_1, a_2 \rangle \\
    r_{22} &= \left \Vert a_2 - r_{12} q_1 \right \Vert
\end{aligned}$$
It turns out we can generalize this as Gram-Schmidt orthogonalization.

### Gram-Schmidt Orthogonalization

As may have been suggestive we can directly construct the arrays $Q$ and $R$ via a process of successive orthogonalization.  We have already shown the first two iterations so lets now consider the $j$th iteration.  

We want to subtract off the components of the vector $a_j$ in the direction of the $q_i$ vectors where $i=1,\ldots,j-1$.  This suggests that we define a vector $v_j$ such that
$$\begin{aligned}
    v_j &= a_j - \langle q_1, a_j \rangle - \langle q_2, a_j \rangle - \quad \cdots \quad- \langle q_{j-1}, a_j \rangle \\
    &= a_j - \sum^{j-1}_{i=1} \langle q_i, a_j \rangle.
\end{aligned}$$

We also need to normalize $v_j$ which allows to define the $j$th column of $Q$ as
$$
    q_j = \frac{a_j - \sum^{j-1}_{i=1} \langle q_i, a_j \rangle}{\left \Vert a_j - \sum^{j-1}_{i=1} \langle q_i, a_j \rangle \right \Vert}.
$$

We can also discern what the entries of $R$ are as we can write the matrix multiplication as the sequence
$$\begin{aligned}
    q_1 &= \frac{a_1}{r_{11}} \\
    q_2 &= \frac{a_2 - r_{12} q_1}{r_{22}} \\
    q_3 &= \frac{a_3 - r_{13} q_1 - r_{23} q_2}{r_{33}} \\
    &\vdots \\
    q_n &= \frac{a_n - \sum^{n-1}_{i=1} r_{in} q_i}{r_{nn}}
\end{aligned}$$
leading us to define
$$
    r_{ij} = \left \{ \begin{aligned}
        &\langle q_i, a_j \rangle & &i \neq j \\
        &\left \Vert a_j - \sum^{j-1}_{i=1} r_{ij} q_i \right \Vert & &i = j
    \end{aligned} \right .
$$

This is called the **classical Gram-Schmidt** iteration.  Turns out that the procedure above is unstable because of rounding errors introduced.

In [ ]:
# Implement Classical Gram-Schmidt Iteration
def classic_GS(A):
    m = A.shape[0]
    n = A.shape[1]
    Q = numpy.empty((m, n))
    R = numpy.zeros((n, n))
    for j in range(n):
        v = A[:, j]
        for i in range(j):
            R[i, j] = numpy.dot(Q[:, i].conjugate(), A[:, j])
            v = v - R[i, j] * Q[:, i]
        R[j, j] = numpy.linalg.norm(v, ord=2)
        Q[:, j] = v / R[j, j]
    return Q, R

A = numpy.array([[12, -51, 4], [6, 167, -68], [-4, 24, -41]], dtype=float)
Q, R = classic_GS(A)
print(A)
print(Q)
print(numpy.dot(Q.transpose(), Q))
print(R)
print(numpy.dot(Q, R) - A)

#### Full vs. Reduced QR

If the original matrix $A \in \mathbb C^{m \times n}$ where $m \ge n$ then we can still define a QR factorization, called the **full QR factorization**, which appends columns full of zeros to $R$ to reproduce the full matrix.
$$
    A = Q R = \begin{bmatrix} Q_1 & Q_2 \end{bmatrix} 
              \begin{bmatrix} R_1 \\
                              0 
              \end{bmatrix} = Q_1 R_1
$$
The factorization $Q_1 R_1$ is called the **reduced** or **thin QR factorization** of $A$.

We require that the additional columns added $Q_2$ are an orthonormal basis that is orthogonal itself to $\text{range}(A)$.  If $A$ is full ranked then $Q_1$ and $Q_2$ provide a basis for $\text{range}(A)$ and $\text{null}(A^\ast)$ respectively.

#### QR Existence and Uniqueness
Two important theorems exist regarding this algorithm which we state without proof:

*Every $A \in \mathbb C^{m \times n}$ with $m \geq n$ has a full QR factorization and therefore a reduced QR factorization.*

*Each $A \in \mathbb C^{m \times n}$ with $m \geq n$ of full rank has a unique reduced QR factorization $A = QR$ with $r_{jj} > 0$.*

#### Gram-Schmidt in Terms of Projections

To start lets rewrite classical Gram-Schmidt as a series of projections:
$$
    q_1 = \frac{P_1 a_1}{||P_1 a_1||}, \quad q_2 = \frac{P_2 a_2}{||P_2 a_2||}, \quad \cdots \quad q_n = \frac{P_n a_n}{||P_n a_n||}
$$
where the $P_i$ are orthogonal projectors onto the $q_1, q_2, \ldots, q_{i-1}$, in other words the complement of $\text{span}(a_1, a_2, \ldots, a_{i-1})$.

How should we construct these projectors?

We saw before that we can easily construct an orthogonal projector onto the complement of space by first constructing the projector onto the space itself via
$$
    \hat{\!Q}_{i-1} = \begin{bmatrix}
           &    &       &        \\
           &    &       &        \\
        q_1 & q_2 & \cdots & q_{i-1} \\ 
           &    &       &        \\
           &    &       &       
    \end{bmatrix}.
$$
Constructing the projection onto the space spanned by $q_1$ through $q_{i-1}$ is then
$$
    \hat{\!P}_{i-1} = \hat{\!Q}_{i-1} \hat{\!Q}^\ast_{i-1}
$$
and therefore the projections in Gram-Schmidt orthogonalization is
$$
    P_{i} = I - \hat{\!P}_{i-1} = I - \hat{\!Q}_{i-1} \hat{\!Q}^\ast_{i-1}.
$$

#### Modified Gram-Schmidt

One problem with the original Gram-Schmidt algorithm is it is not stable numerically.  Instead we can derive a modified method that is more numerically stable.

Recall that the basic piece of the original algorithm was to take the inner product of $a_j$ and all the relevant $q_i$.  Using the rewritten version of Gram-Schmidt in terms of projections we then have

$$
    v_i = P_i a_i.
$$

This projection is of rank $m - (i - 1)$ as we know that the resulting $v_i$ are linearly independent by construction.  The modified version of Gram-Schmidt instead uses projections that are all of rank $m-1$.  To construct this projection remember that we can again construct the complement to a projection and perform the following sequence of projections

$$
    P_i = \hat{\!P}_{q_{i-1}} \hat{\!P}_{q_{i-2}} \cdots \hat{\!P}_{q_{2}} \hat{\!P}_{q_{1}}
$$

where $\hat{\!P}_{q_{i}}$ projects onto the complement of the space spanned by $q_i$.  Note that this performs mathematically the same job as $P_i a_i$ however each of these projectors are of rank $m - 1$.  

This leads to the following set of calculations:

$$\begin{aligned}
    1.\quad v^{(1)}_i &= a_i  \\
    2.\quad v^{(2)}_i &= \hat{\!P}_{q_1} v_i^{(1)} = v^{(1)}_i - q_1 q_1^\ast v^{(1)}_i \\
    3.\quad v^{(3)}_i &= \hat{\!P}_{q_2} v_i^{(2)} = v^{(2)}_i - q_2 q_2^\ast v^{(2)}_i \\
    & \text{  } \vdots & &\\
    i.\quad v^{(i)}_i &= \hat{\!P}_{q_{i-1}} v_i^{(i-1)} =  v_i^{(i-1)} - q_{i-1} q_{i-1}^\ast v^{(i-1)}_i
\end{aligned}$$

The reason why this approach is more stable is that we are not projecting with a possibly arbitrarily low-rank projector, instead we only take projectors that are high-rank.

**Example: Implementation of modified Gram-Schmidt**
Implement the modified Gram-Schmidt algorithm checking to make sure the resulting factorization has the required properties.

In [ ]:
# Implement Modified Gram-Schmidt Iteration
def mod_GS(A):
    m = A.shape[0]
    n = A.shape[1]
    Q = numpy.empty((m, n))
    R = numpy.zeros((n, n))
    v = A.copy()
    for i in range(n):
        R[i, i] = numpy.linalg.norm(v[:, i], ord=2)
        Q[:, i] = v[:, i] / R[i, i]
        for j in range(i + 1, n):
            R[i, j] = numpy.dot(Q[:, i].conjugate(), v[:, j])
            v[:, j] -= R[i, j] * Q[:, i]
    return Q, R

A = numpy.array([[12, -51, 4], [6, 167, -68], [-4, 24, -41]], dtype=float)
print(A)
Q, R = mod_GS(A)
print(R)
print(numpy.dot(Q.transpose(), Q))
print("Modified = ")
print(numpy.dot(Q, R) - A)

### Householder Triangularization

One way to also interpret Gram-Schmidt orthogonalization is as a series of multiplications by upper triangular matrices of the matrix A.  For instance the first step in performing the modified algorithm is to divide through by the norm $r_{11} = ||v_1||$ to give $q_1$:

$$
    \begin{bmatrix}
           &    &       &  \\
           &    &       &  \\
        v_1 & v_2 & v_3 &  \cdots & v_n \\
           &    &       &  \\
           &    &       &       
    \end{bmatrix}
    \begin{bmatrix}
        \frac{1}{r_{11}}   &  &\cdots &       \\
           & 1  &          \\
         &  & \ddots &  \\ 
           &    &  & 1 
    \end{bmatrix} = 
    \begin{bmatrix}
           &    &       &        \\
           &    &       &        \\
        q_1 & v_2 &  v_3 & \cdots & v_n \\ 
           &    &       &        \\
           &    &       &       
    \end{bmatrix}
$$

We can also perform all the step (2) evaluations by also combining the step that projects onto the complement of $q_1$ by add the appropriate values to the entire first row:

$$
    \begin{bmatrix}
           &    &       &  \\
           &    &       &  \\
        v_1 & v_2 &  v_3 & \cdots & v_n \\
           &    &       &  \\
           &    &       &       
    \end{bmatrix}
    \begin{bmatrix}
        \frac{1}{r_{11}}   & -\frac{r_{12}}{r_{11}} & -\frac{r_{13}}{r_{11}} & \cdots      \\
           & 1   &         \\
         &  & \ddots &  \\ 
           &    &  & 1 
    \end{bmatrix} = 
    \begin{bmatrix}
           &    &       &        \\
           &    &       &        \\
        q_1 & v_2^{(2)} & v_3^{(2)} & \cdots & v_n^{(2)} \\ 
           &    &       &        \\
           &    &       &       
    \end{bmatrix}
$$

The next step can then be placed into the second row:
$$
    \begin{bmatrix}
           &    &       &  \\
           &    &       &  \\
        v_1 & v_2 & v_3 & \cdots & v_n \\
           &    &       &  \\
           &    &       &       
    \end{bmatrix}
    \cdot R_1 \cdot
    \begin{bmatrix}
        1 &  &  &  & \\
         & \frac{1}{r_{22}}   & -\frac{r_{23}}{r_{22}} & -\frac{r_{25}}{r_{22}} & \cdots      \\
         &   & 1   &         \\
         &  &  & \ddots &  \\ 
         &    &  & & 1 
    \end{bmatrix} = 
    \begin{bmatrix}
           &    &       &        \\
           &    &       &        \\
        q_1 & q_2 & v_3^{(3)} & \cdots & v_n^{(3)} \\ 
           &    &       &        \\
           &    &       &       
    \end{bmatrix}
$$

If we identify the matrices as $R_1$ for the first case, $R_2$ for the second case and so on we can write the algorithm as

$$
    A \underbrace{R_1R_2 \quad \cdots \quad R_n}_{\hat{R}^{-1}} = \hat{\!Q}.
$$

This view of Gram-Schmidt is called Gram-Schmidt triangularization.

Householder triangularization is similar in spirit.  Instead of multiplying $A$ on the right Householder multiplies $A$ on the left by unitary matrices $Q_k$.  Remember that a unitary matrix (or an orthogonal matrix when strictly real) has as it's inverse it's adjoint (transpose when real) $Q^* = Q^{-1}$ so that $Q^* Q = I$.  We therefore have

$$
    Q_n Q_{n-1} \quad \cdots \quad Q_2 Q_1 A = R
$$

which if we identify $Q_n Q_{n-1} \text{  } \cdots \text{  } Q_2 Q_1 = Q^*$ and note that if $Q = Q^\ast_n Q^\ast_{n-1} \text{  } \cdots \text{  } Q^\ast_2 Q^\ast_1$ then $Q$ is also unitary.  

We can then write this as
$$\begin{aligned}
    Q_n Q_{n-1} \text{  } \cdots \text{  } Q_2 Q_1 A &= R \\
    Q_{n-1} \cdots Q_2 Q_1 A &= Q^\ast_n R \\
    & \text{  } \vdots \\
    A &= Q^\ast_1 Q^\ast_2 \text{  } \cdots \text{  } Q^\ast_{n-1} Q^\ast_n R \\
    A &= Q R
\end{aligned}$$

This was we can think of Householder triangularization as one of introducing zeros into $A$ via orthogonal matrices.

$$
    \begin{bmatrix}
        \text{x} & \text{x} & \text{x} \\
        \text{x} & \text{x} & \text{x} \\
        \text{x} & \text{x} & \text{x} \\
        \text{x} & \text{x} & \text{x} \\
    \end{bmatrix} \overset{Q_1}{\rightarrow}
    \begin{bmatrix}
        \text{x} & \text{x} & \text{x} \\
        0 & \text{x} & \text{x} \\
        0 & \text{x} & \text{x} \\
        0 & \text{x} & \text{x} \\
    \end{bmatrix} \overset{Q_2}{\rightarrow}
    \begin{bmatrix}
        \text{x} & \text{x} & \text{x} \\
        0 & \text{x} & \text{x} \\
        0 & 0 & \text{x} \\
        0 & 0 & \text{x} \\
    \end{bmatrix} \overset{Q_3}{\rightarrow}
    \begin{bmatrix}
        \text{x} & \text{x} & \text{x} \\
        0 & \text{x} & \text{x} \\
        0 & 0 & \text{x} \\
        0 & 0 & 0\\
    \end{bmatrix}
$$

Now the question is how do we construct the $Q_k$.  The construction is usually broken down into a matrix of the form

$$
    Q_k = \begin{bmatrix} I & 0 \\ 0 & F \end{bmatrix}
$$

where $I \in \mathbb C^{k-1 \times k-1}$ identity matrix and $F \in \mathbb C^{m - (k - 1) \times m - (k-1)}$ unitary matrix.  Note that this will leave the rows and columns we have already worked on alone and be unitary.  

To construct $F$ consider the transformation that reflects the vector $x$ over the plane $H$ so that $F x = v = ||x|| e_1$:
![Householder reflection](./images/householder.png)
or mathematically
$$
    x = \begin{bmatrix}
        \text{x} \\
        \text{x} \\
        \text{x} \\
        \text{x}
    \end{bmatrix} \overset{F}{\rightarrow}
    Fx = \begin{bmatrix}
        ||x|| \\
        0 \\
        0 \\
        0
    \end{bmatrix} = ||x|| e_1.
$$

This is of course the effect on only one vector.  Any other vector will be reflected across $H$ (technically a hyperplane) which is orthogonal to $v = ||x|| e_1 - x$.  This has a similar construction as to the projector complements we were working with before.  Consider the projector defined as

$$
    P x = \left (I - \frac{v v^\ast}{v^\ast v}\right) x = x - x \left(\frac{v^\ast x}{v^\ast v} \right),
$$

the complement of a projection in the direction of the vector $v$, in other words in the direction of $H$ above.  Since we actually want to transform $x$ to lie in the direction of $e_1$ we need to go twice as far as just the projection onto $H$.  This allows us to identify the matrix $F$ as

$$
    F = I - 2 \frac{v v^\ast}{v^\ast v}.
$$

There is actually a non-uniqueness to which direction we reflect over since another definition of $\hat{H}$ which is orthogonal to the one we originally choose is available.  For numerical stability purposes we will choose the reflector that is the most different from $x$.  This comes back to having difficulties numerically when the vector $x$ is nearly aligned with $e_1$ and therefore one of the $H$ specification.  By convention the $v$ chosen is defined by

$$
    v = \text{sign}(x_1)||x|| e_1 + x.
$$

In [ ]:
# Implementation of Householder QR Factorization
def householder_QR(A, verbose=False):
    R = A.copy()
    v = numpy.empty(A.shape)
    m, n = A.shape
    for k in range(n):
        x = R[k:, k]
        e1 = numpy.zeros(x.shape)
        e1[0] = 1.0
        v[k:, k] = numpy.sign(x[0]) * numpy.linalg.norm(x, ord=2) * e1 + x
        v[k:, k] = v[k:, k] / numpy.linalg.norm(v[k:, k], ord=2)
        R[k:, k:] -= 2.0 * numpy.dot(numpy.outer(v[k:, k], v[k:, k]), R[k:, k:])

    # Form Q
    m, n = A.shape
    Q = numpy.zeros(A.shape)
    for i in range(n):
        en = numpy.zeros(m)
        en[i] = 1.0
        for j in range(n - 1, -1, -1):
            en[j:m] -= 2.0 * numpy.dot(numpy.outer(v[j:, j], v[j:, j]), en[j:m])
        Q[:, i] = en
        
    return Q, R

A = numpy.array([[12, -51, 4], [6, 167, -68], [-4, 24, -41]], dtype=float)
print("Matrix A = ")
print(A)
m, n = A.shape
Q, R = householder_QR(A, verbose=False)
print("Householder (reduced) Q = ")
print(Q)
print("Householder (full) R = ")
print(R)

print("Check to see if factorization worked...")
print(numpy.abs(A - numpy.dot(Q, R[:n, :n])))

In the above algorithm we do not need to explicitly form the matrix $Q$ to save memory and computation.  If we wanted to for instance solve $A x = b$ we again have

$$\begin{aligned}
    A x &= b \\
    Q R x &= b \\
    Rx &= Q^\ast b.
\end{aligned}$$

This requires then the multiplication $Q^\ast b$.  To do this we just need to recognize that the vectors $v$ we save specify the $Q_k$ so we can form the matrices $F$ and multiply the vector directly with the $v_k$s:

#### Example 1:   Random Matrix QR

Consider a matrix $A$ with a random eigenspace and widely varying eigenvalues.  The values along the diagonal of $R$ gives us some idea of the size of the projections as we go, i.e. the larger the values the less effective we are in constructing othorgonal directions.

In [ ]:
N = 80
U, X = numpy.linalg.qr(numpy.random.random((N, N)))
V, X = numpy.linalg.qr(numpy.random.random((N, N)))
S = numpy.diag(2.0**numpy.arange(-1.0, -(N + 1), -1.0))
A = numpy.dot(U, numpy.dot(S, V))

fig = plt.figure()
axes = fig.add_subplot(1, 1, 1)
Q, R = classic_GS(A)
axes.semilogy(numpy.diag(R), 'bo', label="Classic")
Q, R = mod_GS(A)
axes.semilogy(numpy.diag(R), 'ro', label="Modified")
Q, R = householder_QR(A)
axes.semilogy(numpy.diag(R), 'ko', label="Householder")

axes.set_xlabel("Index")
axes.set_ylabel("$R_{ii}$")
axes.legend(loc=3)
axes.plot(numpy.arange(0, N), numpy.ones(N) * numpy.sqrt(numpy.finfo(float).eps), 'k--')
axes.plot(numpy.arange(0, N), numpy.ones(N) * numpy.finfo(float).eps, 'k--')

plt.show()

#### Example 2:  Comparing Orthogonality

Consider
$$
    A = \begin{bmatrix}
        0.70000 & 0.70711 \\ 0.70001 & 0.70711
    \end{bmatrix}.
$$
Check that the matrix $Q$ is really unitary given this matrix.

In [ ]:
%precision 16

A = numpy.array([[0.7, 0.70711], [0.70001, 0.70711]])

Q, R = classic_GS(A)
print("Classic: ", numpy.linalg.norm(numpy.dot(Q.transpose(), Q) - numpy.eye(2)))

Q, R = mod_GS(A)
print("Modified: ", numpy.linalg.norm(numpy.dot(Q.transpose(), Q) - numpy.eye(2)))

Q, R = householder_QR(A)
print("Housholder:", numpy.linalg.norm(numpy.dot(Q.transpose(), Q) - numpy.eye(2)))

Q, R = numpy.linalg.qr(A)
print("Numpy: ", numpy.linalg.norm(numpy.dot(Q.transpose(), Q) - numpy.eye(2)))

### Applications of QR

#### Solving $Ax = b$ with QR

Suppose we want to solve the system $Ax = b$ where $A \in \mathbb C^{m \times m}$.  See if you can figure out how to use a QR factorization to help with this.

Say we have found the QR factorization of $A$, then
$$\begin{aligned}
    A x &= b \\
    QR x & = b \\
    Q^\ast Q R x &= Q^\ast b \\
    R x &= Q^\ast b.
\end{aligned}$$
Given that $R$ is upper triangular we can use back-substitution to find b.

#### Applications: Least Squares Problems

Least squares problems have already been introduced but lets consider how our QR factorizations might help us.  As before the least squares problem is characterized by wanting to find the $b$ such that $||b - Ax||_2$ is minimized over $x \in \mathbb C^n$.  

Since we are using the $\ell_2$ norm and know this is equivalent to the Euclidean norm we know that there is a geometric interpretation to this goal, find the vector $x$ that gives the minimum distance between the vector $b$ and $A x$.  This can be interpreted as a projection:
![Least-Squares Projection](./images/lsq_projection.png)
where
$$
    r = b - Ax
$$
and
$$
    y = Ax = Pb.
$$
The vector $r$ is called the residual (and the thing we are trying to minimize).  $P$ represents the orthogonal projector onto the $\text{range}(A)$.

QR factorization plays a role similar to the ideas we saw from Householder triangularization.  Define the orthogonal projector $P = Q Q^\ast$ based on the reduced QR factorization of $A$.  We know then that the projector projects onto the span of column space of $A$ ($\text{span}(A)$).  Using this QR factorization we know that the least-squares formulation then becomes
$$\begin{aligned}
    A^\ast A x &= A^\ast b \\
    R^\ast Q^\ast Q R x &= R^\ast Q^\ast b \\
    R^\ast R x & = R^\ast Q^\ast b \\
    R x & = Q^\ast b
\end{aligned}$$
reducing the least-squares calculation to one of finding the QR factorization and backwards substitution.